In [12]:
import os
import GEOparse
import logging
import openai
import pandas as pd
import random

In [6]:
datasets_df = pd.read_excel('Magalhaes_datasets.xlsx', header=1)


In [7]:
datasets_df.head()

,Study,Tissue,Species,Technology
0,AgeMap,Adrenal Glands,Mouse,Microarray
1,AgeMap,Bone Marrow,Mouse,Microarray
2,AgeMap,Brain,Mouse,Microarray
3,AgeMap,Eye,Mouse,Microarray
4,GDS1079,Myoblasts,Mouse,Microarray


In [8]:
gse_id_list = datasets_df.query("Technology=='Microarray' & Study !='AgeMap'").Study.tolist()


In [9]:
gse_id_list

['GDS1079',
 'GDS1278',
 'GDS1279',
 'GDS1280',
 'GDS1280',
 'GDS1311',
 'GDS156',
 'GDS1647',
 'GDS1803',
 'GDS2019',
 'GDS2082',
 'GDS2612',
 'GDS2639',
 'GDS2681',
 'GDS287',
 'GDS288',
 'GDS2912',
 'GDS2929',
 'GDS2972',
 'GDS2973',
 'GDS2996',
 'GDS3077',
 'GDS3102',
 'GDS3102',
 'GDS3182',
 'GDS355',
 'GDS356',
 'GDS3620',
 'GDS3869',
 'GDS3915',
 'GDS3939',
 'GDS3976',
 'GDS399',
 'GDS40',
 'GDS4019',
 'GDS4264',
 'GDS4522',
 'GDS4523',
 'GDS472',
 'GDS473',
 'GDS4804',
 'GDS4858',
 'GDS4874',
 'GDS4892',
 'GDS4904',
 'GDS4925',
 'GDS520',
 'GDS5204',
 'GDS5216',
 'GDS5217',
 'GDS5218',
 'GDS5226',
 'GDS5286',
 'GDS5412',
 'GDS707',
 'GSE11097',
 'GSE11291',
 'GSE11291',
 'GSE11291',
 'GSE12277',
 'GSE12480',
 'GSE16487',
 'GSE19676',
 'GSE21681',
 'GSE27625',
 'GSE32719',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE35322',
 'GSE36192',
 'GSE39540',
 'GSE40645',
 'GSE48911',
 'GSE5086',
 'GSE56045',
 'GSE56580',
 'GSE58137',
 'GSE61915',
 'GSE63060',
 '

In [15]:
gse_id_list = [gse_id for gse_id in gse_id_list if gse_id.startswith('GSE')]


In [16]:
gse_id_list

['GSE11097',
 'GSE11291',
 'GSE11291',
 'GSE11291',
 'GSE12277',
 'GSE12480',
 'GSE16487',
 'GSE19676',
 'GSE21681',
 'GSE27625',
 'GSE32719',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE35322',
 'GSE36192',
 'GSE39540',
 'GSE40645',
 'GSE48911',
 'GSE5086',
 'GSE56045',
 'GSE56580',
 'GSE58137',
 'GSE61915',
 'GSE63060',
 'GSE66857',
 'GSE69187',
 'GSE69832',
 'GSE73450',
 'GSE74463',
 'GSE7958',
 'GSE85084']

In [17]:
sampled_gses = random.sample(gse_id_list, 5)


In [18]:
sampled_gses

['GSE11291', 'GSE61915', 'GSE66857', 'GSE19676', 'GSE16487']